# PulseAD Real-Time Simulation

See how PulseAD works in a production-like scenario, processing data point by point.

**Key concept:** Each detection call uses historical data as context, but only evaluates the latest data point(s) for anomalies. For real-time monitoring, you pass a sliding window of history with each new data point appended at the end.

**Features:**
- Streaming simulation (one datapoint at a time)
- Animated visualization
- GIF export for sharing

In [ ]:
import sys
sys.path.append('../..')

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from datetime import datetime, timedelta
from gradientcast import GradientCastPulseAD

# Replace with your API key
GRADIENTCAST_API_KEY = "your-api-key-here"

ad = GradientCastPulseAD(api_key=GRADIENTCAST_API_KEY)

---
## Generate Simulation Data

Create a realistic time series with injected anomalies.

In [ ]:
def generate_simulation_data(n_points=100, base_value=1500000, anomaly_positions=None):
    """Generate time series data with controlled anomalies."""
    np.random.seed(42)
    
    # Generate timestamps
    start = datetime.now() - timedelta(hours=n_points)
    timestamps = [(start + timedelta(hours=i)).strftime("%m/%d/%Y, %I:%M %p") 
                  for i in range(n_points)]
    
    # Generate values with trend and noise
    trend = np.linspace(0, base_value * 0.1, n_points)
    noise = np.random.normal(0, base_value * 0.03, n_points)
    values = base_value + trend + noise
    
    # Inject anomalies
    if anomaly_positions is None:
        anomaly_positions = [60, 75, 90]  # Default positions
    
    true_anomalies = [False] * n_points
    for pos in anomaly_positions:
        if pos < n_points:
            values[pos] *= 0.6  # 40% drop
            true_anomalies[pos] = True
    
    return timestamps, values.astype(int).tolist(), true_anomalies

# Generate data
timestamps, values, true_anomalies = generate_simulation_data(n_points=80)

print(f"Generated {len(values)} data points")
print(f"Injected anomalies at positions: {[i for i, a in enumerate(true_anomalies) if a]}")

---
## Run Simulation

Process data points one at a time, like in production.

In [ ]:
def run_simulation(timestamps, values, min_history=10):
    """Run anomaly detection simulation.
    
    Simulates real-time processing: for each new data point, we pass
    all history up to that point as context. The detector only evaluates
    the latest point for anomalies, using the history for context.
    """
    results = []
    detected_anomalies = []
    
    for i in range(min_history, len(values)):
        # Build data window: all history up to and including current point
        # Earlier points serve as context; current point (index i) is evaluated
        window_data = [
            {"timestamp": timestamps[j], "value": values[j]}
            for j in range(i + 1)
        ]
        
        # Detect anomalies - context is provided, but only latest point(s) are evaluated
        result = ad.detect({"metric": window_data})
        
        # Check if current point (the last one in window) is anomalous
        is_anomaly = any(
            r.is_anomaly and r.timestamp == timestamps[i] 
            for r in result.results
        )
        
        results.append({
            'index': i,
            'value': values[i],
            'is_anomaly': is_anomaly
        })
        detected_anomalies.append(is_anomaly)
        
        if i % 20 == 0 or is_anomaly:
            status = "ANOMALY" if is_anomaly else "OK"
            print(f"Point {i}: {values[i]:,} [{status}]")
    
    return results, detected_anomalies

# Run simulation
print("Running simulation...\n")
results, detected = run_simulation(timestamps, values)
print(f"\nSimulation complete. Detected {sum(detected)} anomalies.")

---
## Animated Visualization

Watch the detection happen in real-time.

In [ ]:
def create_animation(values, detected_anomalies, min_history=10):
    """Create animated visualization of the simulation."""
    
    fig, ax = plt.subplots(figsize=(12, 5))
    
    # Initialize empty plot elements
    line, = ax.plot([], [], 'b-', lw=2, label='Values')
    scatter_normal = ax.scatter([], [], c='blue', s=30, zorder=4)
    scatter_anomaly = ax.scatter([], [], c='red', s=150, zorder=5, 
                                  edgecolors='darkred', linewidths=2, label='Anomaly')
    
    # Set axis limits
    ax.set_xlim(0, len(values))
    ax.set_ylim(min(values) * 0.9, max(values) * 1.1)
    ax.set_xlabel('Time (hours)')
    ax.set_ylabel('Value')
    ax.set_title('PulseAD Real-Time Anomaly Detection')
    ax.legend(loc='upper left')
    ax.grid(alpha=0.3)
    
    # Prepare data arrays
    x_data = []
    y_data = []
    anomaly_x = []
    anomaly_y = []
    
    def init():
        line.set_data([], [])
        scatter_normal.set_offsets(np.empty((0, 2)))
        scatter_anomaly.set_offsets(np.empty((0, 2)))
        return line, scatter_normal, scatter_anomaly
    
    def update(frame):
        # Add initial history
        if frame < min_history:
            x_data.append(frame)
            y_data.append(values[frame])
        else:
            # Add new point
            x_data.append(frame)
            y_data.append(values[frame])
            
            # Check if anomaly
            result_idx = frame - min_history
            if result_idx < len(detected_anomalies) and detected_anomalies[result_idx]:
                anomaly_x.append(frame)
                anomaly_y.append(values[frame])
        
        # Update line
        line.set_data(x_data, y_data)
        
        # Update scatter plots
        if x_data:
            scatter_normal.set_offsets(np.column_stack([x_data, y_data]))
        if anomaly_x:
            scatter_anomaly.set_offsets(np.column_stack([anomaly_x, anomaly_y]))
        
        return line, scatter_normal, scatter_anomaly
    
    anim = animation.FuncAnimation(
        fig, update, frames=len(values),
        init_func=init, blit=True, interval=100
    )
    
    plt.close(fig)
    return anim

# Create animation
anim = create_animation(values, detected, min_history=10)

# Display in notebook
HTML(anim.to_jshtml())

---
## Save as GIF

In [ ]:
# Save animation as GIF
anim = create_animation(values, detected, min_history=10)
anim.save('pulseAD_simulation.gif', writer='pillow', fps=10)
print("Animation saved as 'pulseAD_simulation.gif'")

---
## Static Summary

In [ ]:
# Final static visualization
plt.figure(figsize=(14, 5))

# Plot all values
plt.plot(values, 'b-', lw=1.5, alpha=0.7, label='Values')

# Mark true anomalies
true_x = [i for i, a in enumerate(true_anomalies) if a]
true_y = [values[i] for i in true_x]
plt.scatter(true_x, true_y, c='red', s=200, zorder=5, 
            edgecolors='darkred', linewidths=2, label='Detected Anomaly')

# Add detection start line
plt.axvline(x=10, color='gray', linestyle='--', alpha=0.5, label='Detection starts')

plt.xlabel('Time (hours)')
plt.ylabel('Value')
plt.title('PulseAD Simulation Results')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# Summary stats
print(f"\nSummary:")
print(f"  Total points processed: {len(values) - 10}")
print(f"  True anomalies: {sum(true_anomalies)}")
print(f"  Detected anomalies: {sum(detected)}")

---
## Key Takeaways

1. **Streaming-compatible**: PulseAD processes data incrementally
2. **Low latency**: Each detection call is fast (~300-500ms)
3. **History matters**: More context generally improves detection

**See also:** [DensityAD Simulation](../densityAD/03_simulation.ipynb) for severity-based detection